In [ ]:
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
    !git clone https://github.com/ai357060/modelF.git
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"
    !git fetch --all
    !git reset --hard origin/master

In [ ]:
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
#     !git clone https://github.com/ai357060/modelF.git
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"
#     !git pull

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import importlib
import tradestats
import pandas as pd
from datetime import datetime

In [ ]:
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
import graphviz

In [ ]:
importlib.reload(tradestats)
from tradestats import *

In [ ]:
prices = loaddata_4h('AUDUSD_Candlestick_4_Hour_BID_03.08.2003-04.09.2021_EEF_GMT.csv')

In [ ]:
rsiKey = [13]
tdiKey = [13]
dictList = []
colFeat = []
keyList = []
# rsiDict   = rsi(prices, rsiKey)
tdiDict   = tdi1(prices, tdiKey)
# dictList.append(rsiDict.df); colFeat.append('rsi'); keyList.append(rsiKey) 
dictList.append(tdiDict.df); colFeat.append('tdi'); keyList.append(tdiKey) 

In [ ]:
# prices.dropna(inplace=True)
masterFrame = pd.DataFrame(index=prices.index)
masterFrame[['id','year','month','day','date','hour','weekday','open','high','low','close','volume']] = prices[['id','year','month','day','date','hour','weekday','open','high','low','close','volume']]   

In [ ]:
for i in range(0,len(dictList)):#iteracja po funkcjach
    for j in keyList[i]: #iteracja po okresach
        for k in list(dictList[i][j]): #iteracja po kolumnach danej funkcji w okresie
            colID = colFeat[i] + str(j) + k
            masterFrame[colID] = dictList[i][j][k]

In [ ]:
# masterFrame.to_csv(sep=';',path_or_buf='../Data/masterFrame.csv',date_format="%Y-%m-%d",index = False,na_rep='')

In [ ]:
masterFrame = masterFrame.dropna()

In [ ]:
# masterFrame.iloc[0:-50,:20]

In [109]:
importlib.reload(tradestats)
from tradestats import *
starttime = datetime.now()
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.003],False)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5,9],[13,17],[0.001,0.002,0.003,0.004,0.005,0.006],True)
alltrades = preparetrades(masterFrame,[1],[5],[13],[0.003],2015,2021)
trades = cleartrades(alltrades.copy(),True)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.002],True)
endtime = datetime.now()
print(str(endtime - starttime))

0:00:01.690380


In [ ]:
#____________________________________________________________________________________________________________________
#
#
trades.head(2)

In [ ]:
X = trades.iloc[:, :-1].values
feature_names = trades.columns[:-1]
y = trades.iloc[:, -1:].values

In [ ]:
# regr = DecisionTreeRegressor(min_samples_leaf=0.001,criterion='mse',max_depth=17, random_state=1234)
# criterion{“mse”, “friedman_mse”, “mae”, “poisson”}, default=”mse”
regr = DecisionTreeClassifier(min_samples_leaf=1,criterion='gini',max_depth=8, random_state=1234)
# criterion{“gini”, “entropy”}, default=”gini”
regr.fit(X, y)
score = regr.score(X, y)
print(score)

In [ ]:
"""
all features mse depth 8 leaf 0.05 =0,0049
wyciąłem poniższe mse depth 8 leaf 0.05 =0,0043
#     df['green1_red1'] = df.green1 - df.red1
#     df['green2_red2'] = df.green2 - df.red2
#     df['green_red_change'] = df.green1_red1 - df.green2_red2
#     df['green_red_slope_change'] = df.green_slope - df.red_slope
all features mse depth 8 leaf 0.1 =0.0028
all features friedman_mse depth 8 leaf 0.05 =0.0049
all features mse depth 8 leaf 0.03 = 0.0069
all features mse depth 8 leaf 0.01 = 0.0074
all features mse depth 8 leaf 1 = 0.048
all features mse depth 9 leaf 1 = 0.051
all features mse depth 15 leaf 1 = 0.11
all features mse depth 15 leaf 0.001 = 0.054
all features mse depth 15 leaf 0.05 = 0,028
all features mse depth 5 leaf 0.05 = 0.0016
"""

In [ ]:
dd = pd.DataFrame()
dd['f'] = feature_names.values
dd['i'] = regr.feature_importances_
dd.sort_values(by=['i'], ascending=False)

In [ ]:
# text_representation = tree.export_text(regr)
# print(text_representation)

In [ ]:
from graphviz import Source
from sklearn import tree
graph = Source( tree.export_graphviz(regr, out_file=None, feature_names=feature_names,rounded = True, proportion = False, 
                 filled = True))
graph.format = 'png'
graph.render('dtree_render',view=True)

In [ ]:
alltrades.head()

In [ ]:
importlib.reload(tradestats)
from tradestats import *
stats = stattrades(alltrades)

In [110]:
#######TEST########
importlib.reload(tradestats)
from tradestats import *
stats = runtrades_v1_4h_1(alltrades)

allexecs:  453600
____progress:   0.2205
elapsed:        0:00:14.229760
last run:       0:00:14.229760
remaining:      1:47:20.389376
estimated end:  2021-10-07 20:21:19.874985
____progress:   0.4409
elapsed:        0:00:30.724997
last run:       0:00:16.495237
remaining:      1:55:37.704323
estimated end:  2021-10-07 20:29:53.685169
____progress:   0.6614
elapsed:        0:00:47.036077
last run:       0:00:16.311080
remaining:      1:57:44.818765
estimated end:  2021-10-07 20:32:17.110691
____progress:   0.8818
elapsed:        0:01:01.785803
last run:       0:00:14.749726
remaining:      1:55:44.724257
estimated end:  2021-10-07 20:30:31.765909
____progress:   1.1023
elapsed:        0:01:17.594102
last run:       0:00:15.808299
remaining:      1:56:01.742831
estimated end:  2021-10-07 20:31:04.592782
____progress:   1.3228
elapsed:        0:01:34.067253
last run:       0:00:16.473151
remaining:      1:56:57.417074
estimated end:  2021-10-07 20:32:16.740176
____progress:   1.5432
elaps

____progress:   11.6843
elapsed:        0:12:44.691053
last run:       0:00:15.874337
remaining:      1:36:19.910110
estimated end:  2021-10-07 20:22:49.857012
____progress:   11.9048
elapsed:        0:12:58.837849
last run:       0:00:14.146796
remaining:      1:36:03.400083
estimated end:  2021-10-07 20:22:47.493781
____progress:   12.1252
elapsed:        0:13:12.373071
last run:       0:00:13.535222
remaining:      1:35:42.543747
estimated end:  2021-10-07 20:22:40.172667
____progress:   12.3457
elapsed:        0:13:26.632751
last run:       0:00:14.259680
remaining:      1:35:27.092532
estimated end:  2021-10-07 20:22:38.987644
____progress:   12.5661
elapsed:        0:13:40.354809
last run:       0:00:13.722058
remaining:      1:35:07.942408
estimated end:  2021-10-07 20:22:33.553066
____progress:   12.7866
elapsed:        0:13:55.154744
last run:       0:00:14.799935
remaining:      1:34:56.331323
estimated end:  2021-10-07 20:22:36.741916
____progress:   13.0071
elapsed:        

____progress:   23.1481
elapsed:        0:25:35.927903
last run:       0:00:13.543162
remaining:      1:24:59.280638
estimated end:  2021-10-07 20:24:20.464390
____progress:   23.3686
elapsed:        0:25:49.596492
last run:       0:00:13.668589
remaining:      1:24:41.506987
estimated end:  2021-10-07 20:24:16.359328
____progress:   23.5891
elapsed:        0:26:03.127972
last run:       0:00:13.531480
remaining:      1:24:23.365935
estimated end:  2021-10-07 20:24:11.749756
____progress:   23.8095
elapsed:        0:26:17.727161
last run:       0:00:14.599189
remaining:      1:24:08.726915
estimated end:  2021-10-07 20:24:11.709925
____progress:   24.0300
elapsed:        0:26:33.895636
last run:       0:00:16.168475
remaining:      1:23:59.049873
estimated end:  2021-10-07 20:24:18.201358
____progress:   24.2504
elapsed:        0:26:50.175062
last run:       0:00:16.279426
remaining:      1:23:49.601375
estimated end:  2021-10-07 20:24:25.032286
____progress:   24.4709
elapsed:        

____progress:   34.6120
elapsed:        0:38:33.101022
last run:       0:00:14.403279
remaining:      1:12:49.845625
estimated end:  2021-10-07 20:25:08.202496
____progress:   34.8325
elapsed:        0:38:50.799309
last run:       0:00:17.698287
remaining:      1:12:40.659973
estimated end:  2021-10-07 20:25:16.715131
____progress:   35.0529
elapsed:        0:39:07.284710
last run:       0:00:16.485401
remaining:      1:12:29.119972
estimated end:  2021-10-07 20:25:21.660531
____progress:   35.2734
elapsed:        0:39:22.730705
last run:       0:00:15.445995
remaining:      1:12:15.610844
estimated end:  2021-10-07 20:25:23.599413
____progress:   35.4938
elapsed:        0:39:39.542646
last run:       0:00:16.809926
remaining:      1:12:04.560113
estimated end:  2021-10-07 20:25:29.358608
____progress:   35.7143
elapsed:        0:39:55.565880
last run:       0:00:16.023234
remaining:      1:11:52.018584
estimated end:  2021-10-07 20:25:32.840313
____progress:   35.9347
elapsed:        

____progress:   46.0758
elapsed:        0:51:16.545389
last run:       0:00:13.092573
remaining:      1:00:00.588527
estimated end:  2021-10-07 20:25:02.389765
____progress:   46.2963
elapsed:        0:51:30.525551
last run:       0:00:13.980162
remaining:      0:59:45.009639
estimated end:  2021-10-07 20:25:00.791039
____progress:   46.5168
elapsed:        0:51:44.152793
last run:       0:00:13.633763
remaining:      0:59:29.040131
estimated end:  2021-10-07 20:24:58.455294
____progress:   46.7372
elapsed:        0:51:57.266984
last run:       0:00:13.107670
remaining:      0:59:12.508035
estimated end:  2021-10-07 20:24:55.030868
____progress:   46.9577
elapsed:        0:52:10.431745
last run:       0:00:13.164761
remaining:      0:58:56.065154
estimated end:  2021-10-07 20:24:51.752748
____progress:   47.1781
elapsed:        0:52:24.982362
last run:       0:00:14.550617
remaining:      0:58:41.204551
estimated end:  2021-10-07 20:24:51.442762
____progress:   47.3986
elapsed:        

____progress:   57.5397
elapsed:        1:06:29.983225
last run:       0:00:16.022735
remaining:      0:49:04.332449
estimated end:  2021-10-07 20:29:19.571523
____progress:   57.7601
elapsed:        1:06:44.518759
last run:       0:00:14.535534
remaining:      0:48:48.495398
estimated end:  2021-10-07 20:29:18.270006
____progress:   57.9806
elapsed:        1:06:59.752829
last run:       0:00:15.234070
remaining:      0:48:33.174484
estimated end:  2021-10-07 20:29:18.183162
____progress:   58.2011
elapsed:        1:07:19.255933
last run:       0:00:19.503104
remaining:      0:48:20.920170
estimated end:  2021-10-07 20:29:25.431952
____progress:   58.4215
elapsed:        1:07:40.013761
last run:       0:00:20.757828
remaining:      0:48:09.504133
estimated end:  2021-10-07 20:29:34.773743
____progress:   58.6420
elapsed:        1:08:00.005362
last run:       0:00:19.991601
remaining:      0:47:57.477466
estimated end:  2021-10-07 20:29:42.738677
____progress:   58.8624
elapsed:        

____progress:   69.0035
elapsed:        1:21:23.199191
last run:       0:00:17.190611
remaining:      0:36:33.539317
estimated end:  2021-10-07 20:31:41.994357
____progress:   69.2240
elapsed:        1:21:39.448723
last run:       0:00:16.249532
remaining:      0:36:18.226248
estimated end:  2021-10-07 20:31:42.930820
____progress:   69.4444
elapsed:        1:21:56.249771
last run:       0:00:16.801048
remaining:      0:36:03.149899
estimated end:  2021-10-07 20:31:44.655519
____progress:   69.6649
elapsed:        1:22:15.983097
last run:       0:00:19.733326
remaining:      0:35:49.339475
estimated end:  2021-10-07 20:31:50.578421
____progress:   69.8854
elapsed:        1:22:30.464932
last run:       0:00:14.482835
remaining:      0:35:33.228737
estimated end:  2021-10-07 20:31:48.950518
____progress:   70.1058
elapsed:        1:22:47.400285
last run:       0:00:16.935351
remaining:      0:35:18.174461
estimated end:  2021-10-07 20:31:50.832591
____progress:   70.3263
elapsed:        

____progress:   80.4674
elapsed:        1:39:26.205208
last run:       0:00:19.907833
remaining:      0:24:08.235018
estimated end:  2021-10-07 20:37:19.698068
____progress:   80.6878
elapsed:        1:39:52.668707
last run:       0:00:26.461506
remaining:      0:23:54.310871
estimated end:  2021-10-07 20:37:32.235427
____progress:   80.9083
elapsed:        1:40:18.760577
last run:       0:00:26.092868
remaining:      0:23:40.230697
estimated end:  2021-10-07 20:37:44.249782
____progress:   81.1287
elapsed:        1:40:37.266383
last run:       0:00:18.503811
remaining:      0:23:24.320659
estimated end:  2021-10-07 20:37:46.842891
____progress:   81.3492
elapsed:        1:40:58.408259
last run:       0:00:21.142860
remaining:      0:23:09.000918
estimated end:  2021-10-07 20:37:52.668004
____progress:   81.5697
elapsed:        1:41:21.385020
last run:       0:00:22.973783
remaining:      0:22:54.064291
estimated end:  2021-10-07 20:38:00.706155
____progress:   81.7901
elapsed:        

____progress:   91.9312
elapsed:        1:58:09.313502
last run:       0:00:18.087160
remaining:      0:10:22.227516
estimated end:  2021-10-07 20:42:16.798859
____progress:   92.1517
elapsed:        1:58:28.919579
last run:       0:00:19.605081
remaining:      0:10:05.448653
estimated end:  2021-10-07 20:42:19.624081
____progress:   92.3721
elapsed:        1:58:46.349795
last run:       0:00:17.431213
remaining:      0:09:48.476618
estimated end:  2021-10-07 20:42:20.083259
____progress:   92.5926
elapsed:        1:59:03.786690
last run:       0:00:17.435898
remaining:      0:09:31.502935
estimated end:  2021-10-07 20:42:20.546471
____progress:   92.8131
elapsed:        1:59:23.496434
last run:       0:00:19.708747
remaining:      0:09:14.703049
estimated end:  2021-10-07 20:42:23.455332
____progress:   93.0335
elapsed:        1:59:42.028025
last run:       0:00:18.531591
remaining:      0:08:57.801151
estimated end:  2021-10-07 20:42:25.085025
____progress:   93.2540
elapsed:        

In [ ]:
#######TEST######## v2
importlib.reload(tradestats)
from tradestats import *
stats = runtrades_v2_4h_0(alltrades)

In [ ]:
importlib.reload(tradestats)
from tradestats import *
stats = runtrades_v2_4h_2(alltrades,1,5,13,10)